<a href="https://colab.research.google.com/github/Abhijitgayen/SNA_As_Msc_projects/blob/main/main_work/LSTM%2BW2V%2BsemEval_2013_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Libary_enter

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [2]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

## Some_function for prepossing

In [3]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['tweet']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    if sentiment == 'neutral':
        return [0.5,0.5]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['tweet'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['label'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

## Data_Set_add

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
tweetsDatas=pd.read_csv('/content/drive/MyDrive/2017_data_set/dataset/train/twitter-2013train-A.txt', sep="\t", header=None, names=["id","label", "tweet"]); #skiping these two rows as they have some bad data
testData=pd.read_csv('/content/drive/MyDrive/2017_data_set/dataset/train/twitter-2013test-A.txt',sep="\t", header=None, names=["id","label", "tweet"]); #test_data

In [23]:
print(tweetsDatas.head())
print(tweetsDatas.label.value_counts())
print(testData.head())
print(testData.label.value_counts())

                   id     label  \
0  264183816548130816  positive   
1  263405084770172928  negative   
2  262163168678248449  negative   
3  264249301910310912  negative   
4  262682041215234048   neutral   

                                               tweet  
0  Gas by my house hit $3.39!!!! I\u2019m going t...  
1  Theo Walcott is still shit\u002c watch Rafa an...  
2  its not that I\u2019m a GSP fan\u002c i just h...  
3  Iranian general says Israel\u2019s Iron Dome c...  
4  Tehran\u002c Mon Amour: Obama Tried to Establi...  
neutral     4586
positive    3640
negative    1458
Name: label, dtype: int64
                   id     label  \
0  264238274963451904  positive   
1  218775148495515649  positive   
2  258965201766998017   neutral   
3  262926411352903682  negative   
4  171874368908050432   neutral   

                                               tweet  
0  @jjuueellzz down in the Atlantic city, ventnor...  
1  Musical awareness: Great Big Beautiful Tomorro...  
2  On 

In [24]:
df=tweetsDatas
# max_length is used for max sequence of input
max_length = get_max_length(df)
print(max_length)

train_X, train_Y = preprocess(df)

35


## ALL_models

In [7]:
# LSTM model
model = Sequential()
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# LSTM model
model = Sequential()
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## train Models

In [25]:
print('Train...')
model.fit(train_X, train_Y,epochs=100)

Train...
Epoch 1/100
303/303 [==============================] - 20s 62ms/step - loss: 0.6720 - accuracy: 0.7874
Epoch 2/100
303/303 [==============================] - 19s 62ms/step - loss: 0.6314 - accuracy: 0.7988
Epoch 3/100
303/303 [==============================] - 19s 61ms/step - loss: 0.6163 - accuracy: 0.7951
Epoch 4/100
303/303 [==============================] - 19s 61ms/step - loss: 0.6083 - accuracy: 0.7928
Epoch 5/100
303/303 [==============================] - 19s 62ms/step - loss: 0.5998 - accuracy: 0.7980
Epoch 6/100
303/303 [==============================] - 19s 63ms/step - loss: 0.5920 - accuracy: 0.7965
Epoch 7/100
303/303 [==============================] - 19s 62ms/step - loss: 0.5859 - accuracy: 0.7887
Epoch 8/100
303/303 [==============================] - 19s 61ms/step - loss: 0.5773 - accuracy: 0.7981
Epoch 9/100
303/303 [==============================] - 18s 61ms/step - loss: 0.5702 - accuracy: 0.7954
Epoch 10/100
303/303 [==============================] - 19s 61ms

In [26]:
test_df=testData
print(test_df.label.value_counts())
max_length = get_max_length(test_df)
test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

neutral     1513
positive    1475
negative     559
Name: label, dtype: int64
111/111 - 4s - loss: 0.9994 - accuracy: 0.6778 - 4s/epoch - 35ms/step
Test score: 0.9994271397590637
Test accuracy: 0.6777558326721191
